In [1]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library('lightgbm')



Attaching package: ‘lightgbm’


The following object is masked from ‘package:xgboost’:

    slice




## Read data and process data labels

In [2]:
time_matrix <- matrix(0,ncol = 3, nrow =4)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [3]:
data=read.csv('Weekly_features_matrix.csv')
data=data[,2:(dim(data)[2]-6)]

In [4]:
dim(data)

[1] 359  17

In [5]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,0,1,0.9939369,6.320946e-11,36.725978,23.3448845,0.9600554,6.8563381,0.12476627,0.9982675,9.788630,-0.004831614,0.06449135,-0.45466261,0.2879798,2179
2,1,0,1,0.9946734,6.529056e-11,34.006351,19.8560557,0.9377499,5.8511863,0.09129043,0.9982275,9.779815,0.063312228,0.16413331,-0.36322842,0.3123634,1710
3,1,0,1,0.9991252,8.840224e-13,44.955510,11.0485829,0.6469218,2.6893773,0.02284238,0.9980612,9.806125,0.051096218,2.84368811,-0.07717025,2.4288344,2178
4,1,0,1,0.4159002,3.556671e-07,5.567640,-13.7061999,0.8181018,2.1703527,0.72989048,0.8936552,4.320394,-0.176902780,0.09272949,-0.55370413,0.3546872,2597
5,1,0,1,0.2103179,9.307235e-06,-12.975059,-1.0188984,0.6064015,0.7164726,0.87229229,0.6882600,1.428560,-0.261675675,0.10934385,-0.57273878,0.3857296,1603
6,1,0,1,0.4338204,1.481087e-06,6.188379,0.3520556,0.5692734,0.6017905,0.78743719,0.7555181,2.865383,-0.265161932,0.11337659,-0.56140364,0.3372221,1602


In [6]:
dlist= load('week_ets_datalist.RData')
datalist=eval(parse(text = dlist ))
res=datalist[[1]]
MASE=res[,,,6]
m=5

In [7]:
dim(MASE)

[1] 359   5   4

In [8]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

meanunique=function(x)
    {
    mean(unique(x))
}

In [9]:
nanum=c()
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    a=apply(count,1,min)
    if(sum(is.na(a))>0)
        {
        nanum=append(nanum,i)
    }
    }

In [10]:
nanum

NULL

In [11]:
realbestmin=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    min_value=apply(count,1,min)
    if (max(min_value,na.rm = TRUE)==0){
        realbestmin[i,]=0
        }
    else
        {
        min_value[is.na(min_value)]=100 
        realbestmin[i,]= whichmin(min_value)
    }
    }

In [12]:
table(realbestmin)

realbestmin
  0   1   2   3   4 
 67  31  73  61 127 

In [13]:
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    mean_value=apply(count,1,meanunique)
    if (max(mean_value,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        mean_value[is.na(mean_value)]=100 
        realbestmean[i,]= whichmin(mean_value)
    }
    }

In [14]:
realbestmean

4
4
4
2
0
0
0
2
2
2
3


In [15]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,0,1,0.9939369,6.320946e-11,36.725978,23.3448845,0.9600554,6.8563381,0.12476627,0.9982675,9.788630,-0.004831614,0.06449135,-0.45466261,0.2879798,2179
2,1,0,1,0.9946734,6.529056e-11,34.006351,19.8560557,0.9377499,5.8511863,0.09129043,0.9982275,9.779815,0.063312228,0.16413331,-0.36322842,0.3123634,1710
3,1,0,1,0.9991252,8.840224e-13,44.955510,11.0485829,0.6469218,2.6893773,0.02284238,0.9980612,9.806125,0.051096218,2.84368811,-0.07717025,2.4288344,2178
4,1,0,1,0.4159002,3.556671e-07,5.567640,-13.7061999,0.8181018,2.1703527,0.72989048,0.8936552,4.320394,-0.176902780,0.09272949,-0.55370413,0.3546872,2597
5,1,0,1,0.2103179,9.307235e-06,-12.975059,-1.0188984,0.6064015,0.7164726,0.87229229,0.6882600,1.428560,-0.261675675,0.10934385,-0.57273878,0.3857296,1603
6,1,0,1,0.4338204,1.481087e-06,6.188379,0.3520556,0.5692734,0.6017905,0.78743719,0.7555181,2.865383,-0.265161932,0.11337659,-0.56140364,0.3372221,1602


In [16]:
set.seed(100)
index = sample(2,nrow(data),replace = TRUE,prob=c(0.7,0.3))

In [17]:
train_data=data[index==1,]
test_data=data[index==2,]
train_label_min=realbestmin[index==1,]
test_label_min=realbestmin[index==2,]
train_label_mean=realbestmean[index==1,]
test_label_mean=realbestmean[index==2,]

In [18]:
head(train_data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,0,1,0.9939369,6.320946e-11,36.725978,23.3448845,0.9600554,6.8563381,0.12476627,0.9982675,9.788630,-0.004831614,0.06449135,-0.45466261,0.2879798,2179
2,1,0,1,0.9946734,6.529056e-11,34.006351,19.8560557,0.9377499,5.8511863,0.09129043,0.9982275,9.779815,0.063312228,0.16413331,-0.36322842,0.3123634,1710
3,1,0,1,0.9991252,8.840224e-13,44.955510,11.0485829,0.6469218,2.6893773,0.02284238,0.9980612,9.806125,0.051096218,2.84368811,-0.07717025,2.4288344,2178
4,1,0,1,0.4159002,3.556671e-07,5.567640,-13.7061999,0.8181018,2.1703527,0.72989048,0.8936552,4.320394,-0.176902780,0.09272949,-0.55370413,0.3546872,2597
5,1,0,1,0.2103179,9.307235e-06,-12.975059,-1.0188984,0.6064015,0.7164726,0.87229229,0.6882600,1.428560,-0.261675675,0.10934385,-0.57273878,0.3857296,1603
6,1,0,1,0.4338204,1.481087e-06,6.188379,0.3520556,0.5692734,0.6017905,0.78743719,0.7555181,2.865383,-0.265161932,0.11337659,-0.56140364,0.3372221,1602


In [19]:
end_time = Sys.time()

In [20]:
time_matrix[1,]=end_time-start_time

In [21]:
end_time-start_time

Time difference of 5.387122 secs

## Target the interval where the actual error is minimum

In [22]:
start_time = Sys.time()

In [23]:
dtrain_xg_min_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_min)) 
dtrain_xg_min_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)) )

In [24]:
dtrain_lg_min_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_min))
dtrain_lg_min_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)))

In [ ]:
xgb_min_reg <- xgboost(data = dtrain_xg_min_reg, nround=100)

In [ ]:
xgb_min_cl <- xgboost(data = dtrain_xg_min_cl, nround=100, objective='multi:softmax',num_class=5)

In [ ]:
lgb_min_reg <- lgb.train(data = dtrain_lg_min_reg, nrounds = 100)

In [ ]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_min_cl <- lgb.train(data = dtrain_lg_min_cl,nrounds = 100,params=params)

In [ ]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

In [ ]:
end_time-start_time

## Target the interval where the average error is minimum

In [22]:
start_time = Sys.time()

In [23]:
dtrain_xg_mean_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_mean)) 
dtrain_xg_mean_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)) )

In [24]:
dtrain_lg_mean_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_mean))
dtrain_lg_mean_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)))

In [25]:
xgb_mean_reg <- xgboost(data = dtrain_xg_mean_reg, nround=100)

[1]	train-rmse:1.696854 
[2]	train-rmse:1.282137 
[3]	train-rmse:1.002652 
[4]	train-rmse:0.811794 
[5]	train-rmse:0.651228 
[6]	train-rmse:0.538724 
[7]	train-rmse:0.462025 
[8]	train-rmse:0.408611 
[9]	train-rmse:0.343758 
[10]	train-rmse:0.297571 
[11]	train-rmse:0.255718 
[12]	train-rmse:0.229907 
[13]	train-rmse:0.207861 
[14]	train-rmse:0.192415 
[15]	train-rmse:0.174109 
[16]	train-rmse:0.167939 
[17]	train-rmse:0.151088 
[18]	train-rmse:0.142935 
[19]	train-rmse:0.136564 
[20]	train-rmse:0.126171 
[21]	train-rmse:0.119781 
[22]	train-rmse:0.113731 
[23]	train-rmse:0.104262 
[24]	train-rmse:0.102220 
[25]	train-rmse:0.099280 
[26]	train-rmse:0.089343 
[27]	train-rmse:0.082434 
[28]	train-rmse:0.076344 
[29]	train-rmse:0.074347 
[30]	train-rmse:0.070675 
[31]	train-rmse:0.065481 
[32]	train-rmse:0.057885 
[33]	train-rmse:0.053204 
[34]	train-rmse:0.048630 
[35]	train-rmse:0.047682 
[36]	train-rmse:0.045266 
[37]	train-rmse:0.042161 
[38]	train-rmse:0.037400 
[39]	train-rmse:0.033

In [26]:
xgb_mean_cl <- xgboost(data = dtrain_xg_mean_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.284220 
[2]	train-mlogloss:1.039681 
[3]	train-mlogloss:0.857754 
[4]	train-mlogloss:0.724170 
[5]	train-mlogloss:0.617327 
[6]	train-mlogloss:0.530700 
[7]	train-mlogloss:0.462169 
[8]	train-mlogloss:0.402236 
[9]	train-mlogloss:0.357935 
[10]	train-mlogloss:0.316309 
[11]	train-mlogloss:0.277905 
[12]	train-mlogloss:0.250277 
[13]	train-mlogloss:0.223671 
[14]	train-mlogloss:0.201279 
[15]	train-mlogloss:0.182130 
[16]	train-mlogloss:0.167235 
[17]	train-mlogloss:0.153808 
[18]	train-mlogloss:0.141162 
[19]	train-mlogloss:0.128984 
[20]	train-mlogloss:0.119307 
[21]	train-mlogloss:0.110506 
[22]	train-mlogloss:0.103479 
[23]	train-mlogloss:0.096128 
[24]	train-mlogloss:0.090179 
[25]	train-mlogloss:0.085256 
[26]	train-mlogloss:0.080500 
[27]	train-mlogloss:0.076424 
[28]	train-mlogloss:0.073012 
[29]	train-mlogloss:0.069640 
[30]	train-mlogloss:0.066623 
[31]	train-mlogloss:0.063821 
[32]	train-mlogloss:0.061517 
[33]	train-mlogloss:0.059206 
[34]	train-mlogloss

In [27]:
lgb_mean_reg <- lgb.train(data = dtrain_lg_mean_reg,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.243069 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1132
[LightGBM] [Info] Number of data points in the train set: 251, number of used features: 14
[LightGBM] [Info] Start training from score 2.250996
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [28]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_mean_cl <- lgb.train(data = dtrain_lg_mean_cl,params=params,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.218606 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1132
[LightGBM] [Info] Number of data points in the train set: 251, number of used features: 14
[LightGBM] [Info] Start training from score -1.696812
[LightGBM] [Info] Start training from score -2.091466
[LightGBM] [Info] Start training from score -1.414579
[LightGBM] [Info] Start training from score -1.836573
[LightGBM] [Info] Start training from score -1.234993
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [29]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

## predict

In [ ]:
start_time = Sys.time()

In [ ]:
alldatalgb <- lgb.Dataset(data = as.matrix(data))
alldataxgb <- xgb.DMatrix(data = as.matrix(data))


In [ ]:
xgbregmin=predict(xgb_min_reg,alldataxgb)
xgbclsmin=predict(xgb_min_cl,alldataxgb)
lgbregmin=predict(lgb_min_reg,as.matrix(data))
lgbclsmin=predict(lgb_min_cl,as.matrix(data))

In [ ]:
xgbregmin

In [ ]:
xgbclsmin

In [ ]:
lgbregmin

In [ ]:
lgbclsmin

In [ ]:
datalength=dim(data)[1]
lgbclsm=matrix(lgbclsmin,m,datalength)
lgbclsminr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsminr[i,]=which.max(lgbclsm[,i])
}
lgbclsminr=lgbclsminr-1

In [ ]:
lgbclsminr

In [ ]:
xgbregmean=predict(xgb_mean_reg,alldataxgb)
xgbclsmean=predict(xgb_mean_cl,alldataxgb)
lgbregmean=predict(lgb_mean_reg,as.matrix(data))
lgbclsmean=predict(lgb_mean_cl,as.matrix(data))

In [ ]:
lgbclsm=matrix(lgbclsmean,m,datalength)
lgbclsmeanr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsmeanr[i,]=which.max(lgbclsm[,i])
}
lgbclsmeanr=lgbclsmeanr-1

In [ ]:
preallmin=cbind(xgbclsmin,xgbregmin)
preallmin=cbind(preallmin,lgbclsminr)
preallmin=cbind(preallmin,lgbregmin)

In [ ]:
colnames(preallmin)=c('xgbclsmin','xgbregmin','lgbclsmin','lgbregmin')
head(preallmin)

In [ ]:
preallmean=cbind(xgbclsmean,xgbregmean)
preallmean=cbind(preallmean,lgbclsmeanr)
preallmean=cbind(preallmean,lgbregmean)

In [ ]:
head(preallmean)

In [ ]:
colnames(preallmean)=c('xgbclsmean','xgbregmean','lgbclsmean','lgbregmean')
head(preallmean)

In [ ]:
end_time = Sys.time()
time_matrix[4,]=end_time-start_time

In [ ]:
result_list=list(preallmin,preallmean,time_matrix)

In [ ]:
save(result_list, file = "weekly_ets_opt_pre_result.RData")

In [30]:
time_matrix

user_time,system_time,elapsed_time
5.387122,5.387122,5.387122
0.000000,0.000000,0.000000
40.366545,40.366545,40.366545
0.000000,0.000000,0.000000


In [31]:
importance_matrix1 <- xgb.importance(model = xgb_mean_cl)
importance_matrix2 <- xgb.importance(model = xgb_mean_reg)
importance_matrix3 <- lgb.importance(lgb_mean_cl)
importance_matrix4 <- lgb.importance(lgb_mean_reg)

importance_matrix1=importance_matrix1[order(importance_matrix1$Feature),]
importance_matrix2=importance_matrix2[order(importance_matrix2$Feature),]
importance_matrix3=importance_matrix3[order(importance_matrix3$Feature),]
importance_matrix4=importance_matrix4[order(importance_matrix4$Feature),]

importance_matrix_mean=(importance_matrix1[,2:4]+importance_matrix2[,2:4]+importance_matrix3[,2:4]+importance_matrix4[,2:4])/4

importance=cbind(importance_matrix1[,1],importance_matrix_mean)

write.csv(importance,'w_ets_imp_mean.csv')